<a href="https://colab.research.google.com/github/ShaunakSen/Data-Science-and-Machine-Learning/blob/master/Hyperopt_A_Conceptual_Explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A Conceptual Explanation of Bayesian Hyperparameter Optimization for Machine Learning


> Based on the article by Will Koehrsen: https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

---

Following are four common methods of hyperparameter optimization for machine learning in order of increasing efficiency:

1. Manual
2. Grid search
3. Random search
4. Bayesian model based opt

![](https://miro.medium.com/max/2000/1*E0_THdPH2NfKB37JUQB8Eg.png)

Validation Errors comparing random search and a model based approach on LFW (left) and PubFig83 (right)

These figures compare validation error for hyperparameter optimization of an image classification neural network with random search in grey and Bayesian Optimization (using the Tree Parzen Estimator or TPE) in green. Lower is better: a smaller validation set error generally means better test set performance, and a smaller number of trials means less time invested. Clearly, there are significant advantages to Bayesian methods, and these graphs, along with other impressive results, convinced me it was time to take the next step and learn model-based hyperparameter optimization.

> The one-sentence summary of Bayesian hyperparameter optimization is: build a probability model of the objective function and use it to select the most promising hyperparameters to evaluate in the true objective function.

### Hyperparameter Optimization

The aim of hyperparameter optimization in machine learning is to find the hyperparameters of a given machine learning algorithm that return the best performance as measured on a validation set. (Hyperparameters, in contrast to model parameters, are set by the machine learning engineer before training. The number of trees in a random forest is a hyperparameter while the weights in a neural network are model parameters learned during training. I like to think of hyperparameters as the model settings to be tuned.)


Hyperparameter optimization is represented in equation form as:

![](https://miro.medium.com/max/780/1*QR4_VOfAAWLVe2I0nqwtTg.png)

Here f(x) represents an objective score to minimize— such as RMSE or error rate— evaluated on the validation set; x* is the set of hyperparameters that yields the lowest value of the score, and x can take on any value in the domain X. **In simple terms, we want to find the model hyperparameters that yield the best score on the validation set metric.**

The problem with hyperparameter optimization is that evaluating the objective function to find the score is extremely expensive. Each time we try different hyperparameters, we have to train a model on the training data, make predictions on the validation data, and then calculate the validation metric. With a large number of hyperparameters and complex models such as ensembles or deep neural networks that can take days to train, this process quickly becomes intractable to do by hand!

Grid search and random search are slightly better than manual tuning because we set up a grid of model hyperparameters and run the train-predict -evaluate cycle automatically in a loop while we do more productive things (like feature engineering). However, even these methods are relatively inefficient because they do not choose the next hyperparameters to evaluate based on previous results. **Grid and random search are completely uninformed by past evaluations, and as a result, often spend a significant amount of time evaluating “bad” hyperparameters.**

For example, if we have the following graph with a lower score being better, where does it make sense to concentrate our search? If you said below 200 estimators, then you already have the idea of Bayesian optimization! We want to focus on the most promising hyperparameters, and if we have a record of evaluations, then it makes sense to use this information for our next choice.

![](https://miro.medium.com/max/1120/1*MiNXGrkk5BbjfkNAXZQSNA.png)

Random and grid search pay no attention to past results at all and would keep searching across the entire range of the number of estimators even though it’s clear the optimal answer (probably) lies in a small region!






### Bayesian Optimization

Bayesian approaches, in contrast to random or grid search, keep track of past evaluation results which they use to form a probabilistic model mapping hyperparameters to a probability of a score on the objective function:

![](https://miro.medium.com/max/614/1*u00KlxHhd1fz6-Jaeou6PA.png)

In the literature, this model is called a “surrogate” for the objective function and is represented as p(y | x). The surrogate is much easier to optimize than the objective function and Bayesian methods work by finding the next set of hyperparameters to evaluate on the actual objective function by selecting hyperparameters that perform best on the surrogate function. In other words:


1. Build a surrogate probability model of the objective function
2. Find the hyperparameters that perform best on the surrogate
3. Apply these hyperparameters to the true objective function
4. Update the surrogate model incorporating the new results
5. Repeat steps 2–4 until max iterations or time is reached


The aim of Bayesian reasoning is to become “less wrong” with more data which these approaches do by continually updating the surrogate probability model after each evaluation of the objective function.

At a high-level, Bayesian optimization methods are efficient because they choose the next hyperparameters in an informed manner. The basic idea is: **spend a little more time selecting the next hyperparameters in order to make fewer calls to the objective function**. In practice, the time spent selecting the next hyperparameters is inconsequential compared to the time spent in the objective function. By evaluating hyperparameters that appear more promising from past results, Bayesian methods can find better model settings than random search in fewer iterations.

If there’s one thing to take away from this article it’s that Bayesian model-based methods can find better hyperparameters in less time because they reason about the best set of hyperparameters to evaluate based on past trials.

As a good visual description of what is occurring in Bayesian Optimization take a look at the images below (source). The first shows an initial estimate of the surrogate model — in black with associated uncertainty in gray — after two evaluations. Clearly, the surrogate model is a poor approximation of the actual objective function in red:

![](https://miro.medium.com/max/1400/1*RQ-pAwQ88yC904QppChGPQ.png)

The next image shows the surrogate function after 8 evaluations. Now the surrogate almost exactly matches the true function. Therefore, if the algorithm selects the hyperparameters that maximize the surrogate, they will likely yield very good results on the true evaluation function.

![](https://miro.medium.com/max/1400/1*bSLAe1LCj3mMKfaZsQWCrw.png)

Bayesian methods have always made sense to me because they operate in much the same way we do: we form an initial view of the world (called a prior) and then we update our model based on new experiences (the updated model is called a posterior). Bayesian hyperparameter optimization takes that framework and applies it to finding the best value of model settings!


### Sequential Model-Based Optimization

Sequential model-based optimization (SMBO) methods (SMBO) are a formalization of Bayesian optimization. The sequential refers to running trials one after another, each time trying better hyperparameters by applying Bayesian reasoning and updating a probability model (surrogate).


There are five aspects of model-based hyperparameter optimization:


1. A domain of hyperparameters over which to search (using hp.search_space)
2. An objective function which takes in hyperparameters and outputs a score that we want to minimize (or maximize) (we define this)
3. **The surrogate model of the objective function**
4. **A criteria, called a selection function, for evaluating which hyperparameters to choose next from the surrogate model** (steps 3 and 4 are what essentially goes under the hood)
5. A history consisting of (score, hyperparameter) pairs used by the algorithm to update the surrogate model (hp trials object)

There are several variants of SMBO methods **that differ in steps 3–4**, namely, how they build a surrogate of the objective function and the criteria used to select the next hyperparameters. Several common choices for the surrogate model are **Gaussian Processes, Random Forest Regressions, and Tree Parzen Estimators (TPE)** while the most common choice for step 4 is **Expected Improvement**. In this post, we will focus on TPE and Expected Improvement.

#### Domain

In the case of random search and grid search, the domain of hyperparameters we search is a grid. An example for a random forest is shown below:

``` python

hyperparameter_grid = {
    'n_estimators': [100, 200, 300, 400, 500, 600],
    'max_depth': [2, 5, 10, 15, 20, 25, 30, 35, 40],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8]
}
```

For a model-based approach, the domain consists of probability distributions. As with a grid, *this lets us encode domain knowledge into the search process by placing greater probability in regions where we think the true best hyperparameters lie*. If we wanted to express the above grid as a probability distribution, it may look something like this:

![](https://miro.medium.com/max/552/1*luY6Ahh7uttR4quIcgOCBw.png)

![](https://miro.medium.com/max/552/1*YfoPLKK8_WXIsRaQ7zcSjg.png)

![](https://miro.medium.com/max/552/1*e6cIETdFd1rzD9ivofNJqw.png)

> Here we have a uniform, log-normal, and normal distribution. These are informed by prior practice/knowledge (for example the learning rate domain is usually a log-normal distribution over several orders of magnitude).




#### Objective Function

The objective function takes in hyperparameters and outputs a single real-valued score that we want to minimize (or maximize). As an example, let’s consider the case of building a random forest for a regression problem. The hyperparameters we want to optimize are shown in the hyperparameter grid above and the score to minimize is the Root Mean Squared Error. Our objective function would then look like (in Python):

``` python

import numpy as np
from sklearn.ensemble improt RandomForestRegressor

# Function mapping hyperparameters to a real-valued scpre
def objective(hyperparameters):
    
    # Machine learning model
    rf = RandomForestRegressor(**hyperparameters)
    
    # Training 
    rf.fit(X_train, y_train)
    
    # Making predictions and evaluating
    predictions = rf.predict(X_valid)
    rmse = np.sqrt(np.mean(np.square(prediction - y_valid)))
    
    return rmse

```

> While the objective function looks simple, it is very expensive to compute! 

If the objective function could be quickly calculated, then we could try every single possible hyperparameter combination (like in grid search). If we are using a simple model, a small hyperparameter grid, and a small dataset, then this might be the best way to go. However, in cases where the objective function may take hours or even days to evaluate, we want to limit calls to it.

The entire concept of Bayesian model-based optimization is to reduce the number of times the objective function needs to be run by choosing only the most promising set of hyperparameters to evaluate based on previous calls to the evaluation function. The next set of hyperparameters are selected based on a model of the objective function called a surrogate.

#### Surrogate Function (Probability Model)

The surrogate function, also called the response surface, is the probability representation of the objective function built using previous evaluations. This is called sometimes called a response surface because it is a high-dimensional mapping of hyperparameters to the probability of a score on the objective function. Below is a simple example with only two hyperparameters:

![](https://miro.medium.com/max/960/0*aBsprZzniYMB0KWc.png)

There are several different forms of the surrogate function including Gaussian Processes and Random Forest regression. However, in this post we will focus on the Tree-structured Parzen Estimator as put forward by Bergstra et al in the paper “Algorithms for Hyper-Parameter Optimization”. These methods differ in how they construct the surrogate function which we’ll explain in just a bit. First we need to talk about the selection function.


#### Selection Function


The selection function is the criteria by which the next set of hyperparameters are chosen **from the surrogate function**. The most common choice of criteria is **Expected Improvement**:


![](https://miro.medium.com/max/716/1*ebsqjhOTSGKBbIR_RLkjSQ.png)

- Here y* is a threshold value of the objective function
- x is the proposed set of hyperparameters
- y is the actual value of the objective function using hyperparameters x
- p(y | x) is the surrogate probability model expressing the probability of y given x

In simpler terms, the aim is to maximize the Expected Improvement with respect to x. This means finding the best hyperparameters under the surrogate function p (y | x).

We are tryinh to minimize y (say RMSE) and we have a threshold value of y which we call `y*`. 

If p (y | x) is zero everywhere that `y < y*`, then the hyperparameters x are not expected to yield any improvement.  If the integral is positive (that means expected improvement is posituve), then it means that the hyperparameters x are expected to yield a better result **than the threshold value**.


#### Tree-structured Parzen Estimator (TPE)

Now let’s get back to the surrogate function. The methods of SMBO differ in how they construct the surrogate model p(y | x). The Tree-structured Parzen Estimator builds a model by applying Bayes rule. Instead of directly representing p( y | x), it instead uses:

![](https://miro.medium.com/max/256/1*4D1QpDZzWpBOl7ANBhsSJA.png)

p (x | y), which is the probability of the hyperparameters given the score on the objective function, in turn is expressed:

![](https://miro.medium.com/max/582/1*idWxsGylqq2ZaMGpHmbxDg.png)

where y < y* represents a lower value of the objective function than the threshold. The explanation of this equation is that we make two different distributions for the hyperparameters: one where the value of the objective function is less than the threshold, l(x), and one where the value of the objective function is greater than the threshold, g(x).

Let’s update our Random Forest graph to include a threshold:

![](https://miro.medium.com/max/1120/1*H5pyf3G115WGJwPpg65yaQ.png)

Now we construct two probability distributions for the number of estimators, one using the estimators that yielded values under the threshold and one using the estimators that yielded values above the threshold.

![](https://miro.medium.com/max/1160/1*6SH5O_ail54karro8j0NGg.png)

Intuitively, it seems that we want to draw values of x from l(x) and not from g(x) because this distribution is based only on values of x that yielded lower scores than the threshold. It turns out this is exactly what the math says as well! With Bayes Rule, and a few substitutions, the expected improvement equation (which we are trying to maximize) becomes:

![](https://miro.medium.com/max/1246/1*ybiePL_8lKNouHlSb5OSgQ.png)

> The term on the far right is the most important part. What this says is that the Expected Improvement is proportional to the ratio l(x) / g(x) and therefore, to maximize the Expected Improvement, we should maximize this ratio. Our intuition was correct: we should draw values of the hyperparameters which are more likely under l(x) than under g(x)!

The Tree-structured Parzen Estimator works by drawing sample hyperparameters from l(x), evaluating them in terms of l(x) / g(x), and returning the set that yields the highest value under l(x) / g(x) corresponding to the greatest expected improvement. These hyperparameters are then evaluated on the objective function. If the surrogate function is correct, then these hyperparameters should yield a better value when evaluated!

The expected improvement criteria allows the model to balance *exploration versus exploitation*. l(x) is a distribution and not a single value which means that the hyperparameters drawn are likely close but not exactly at the maximum of the expected improvement. Moreover, because the surrogate is just an estimate of the objective function, the selected hyperparameters may not actually yield an improvement when evaluated and the surrogate model will have to be updated. This updating is done based on the current surrogate model and the history of objective function evaluations.

#### Exploration versus exploitation

In probability theory, the multi-armed bandit problem (sometimes called the K- or N-armed bandit problem) is a problem in which a fixed limited set of resources must be allocated between competing (alternative) choices in a way that maximizes their expected gain, when each choice's properties are only partially known at the time of allocation, and may become better understood as time passes or by allocating resources to the choice.[ This is a classic reinforcement learning problem that exemplifies the exploration–exploitation tradeoff dilemma. The name comes from imagining a gambler at a row of slot machines (sometimes known as "one-armed bandits"), who has to decide which machines to play, how many times to play each machine and in which order to play them, and whether to continue with the current machine or try a different machine. The multi-armed bandit problem also falls into the broad category of stochastic scheduling.

In the problem, each machine provides a random reward from a probability distribution specific to that machine. The objective of the gambler is to maximize the sum of rewards earned through a sequence of lever pulls.[ The crucial tradeoff the gambler faces at each trial is between "exploitation" of the machine that has the highest expected payoff and "exploration" to get more information about the expected payoffs of the other machines. The trade-off between exploration and exploitation is also faced in machine learning
